# Let's calculate some winners!

In [12]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [67]:
# ignore an annoying deprecation notice for errors='ignore' setting in pandas 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# imports
import pandas as pd
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.oauth2 import service_account

# Setup Google Sheets credentials (service acct key saved in local file)
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly', 'https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'rubber-ducks-book-club-d1ff42f7cbd8.json'

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

service = build('sheets', 'v4', credentials=credentials)
sheet = service.spreadsheets()



In [76]:
# Fetch vote data
SPREADSHEET_ID = '1C2TuZrF9KFcqwZFIEHbCFDLL1k_T6DkwOGk7yw6gRN4'
VOTES = 'votes'; # votes: books are columns with votes 1-7 in rows

result = sheet.values().get(
    spreadsheetId=SPREADSHEET_ID,
    range=VOTES).execute()
votes = result.get('values', [])

# Convert to pandas DataFrame
df = pd.DataFrame(votes[1:], columns=votes[0])
# Convert votes to integers
df = df.apply(pd.to_numeric, errors='ignore')
 # print it to make sure it went ok
df

,the-most-h,playground,bowling-al,the-upswin,on-tyranny,greek-less,women!-in!,lexicon,apocalypti,self-compa,...,exit-inter,wanderers,gideon-the,the-only-w,"tomorrow,-",algorithms,exhalation,an-absolut,sisters-of,to-shape-a
0,1,6,3,3,6,7,6,6,5,6,...,6,4,7,3,5,3,5,5,6,7
1,6,6,6,7,5,5,2,2,5,7,...,5,2,1,6,4,7,5,2,6,1
2,6,4,4,3,3,5,5,4,5,6,...,6,7,7,7,7,7,5,6,5,4
3,4,4,7,7,7,7,7,4,7,7,...,4,4,7,7,4,4,7,1,7,7
4,6,7,5,5,3,4,6,5,5,1,...,2,6,2,5,3,3,5,6,1,2
5,6,5,4,3,5,2,1,1,1,5,...,3,4,3,6,3,6,3,2,3,2
6,5,6,7,7,5,5,5,5,5,5,...,3,4,4,3,7,7,5,7,4,6
7,3,5,4,7,7,3,5,5,6,6,...,5,5,7,3,2,5,4,3,3,3
8,6,5,5,5,4,6,6,3,7,3,...,2,2,2,7,4,5,6,3,2,5
9,5,6,4,7,5,7,7,7,7,1,...,6,5,1,4,7,6,6,6,5,1


In [90]:
# Simple ranking from highest to lowest based on votes added up together

# Calculate the sum of votes for each book (column)
vote_sums = df.sum()

# Rank the books from highest to lowest based on the sum of votes
ranked_books = vote_sums.sort_values(ascending=False)

# Display the ranked books
print("All books ranked by vote sums", ranked_books)

All books ranked by vote sums the-upswin    71
playground    67
the-only-w    67
algorithms    67
mutual-aid    67
on-tyranny    65
women!-in!    64
exhalation    64
tomorrow,-    63
the-most-h    62
bowling-al    62
greek-less    62
apocalypti    62
gideon-the    61
land          59
wanderers     59
self-compa    58
an-absolut    56
sisters-of    56
the-three-    55
exit-inter    54
hannah-cou    51
harry's-tr    51
we-came-to    50
to-shape-a    50
lexicon       49
dtype: int64


In [91]:
# This is the results ranked just my the highest sum
print("Top 10 ranked by sum", ranked_books.head(10))

Top 10 ranked by sum the-upswin    71
playground    67
the-only-w    67
algorithms    67
mutual-aid    67
on-tyranny    65
women!-in!    64
exhalation    64
tomorrow,-    63
the-most-h    62
dtype: int64


In [92]:
# Sort by number of voters that rated it above 4 ("meh")
votes_above_4 = (df > 4).sum().sort_values(ascending=False)
print("Votes above 4 for each book:\n", votes_above_4)


Votes above 4 for each book:
 the-upswin    10
women!-in!    10
apocalypti    10
algorithms     9
on-tyranny     9
exhalation     9
playground     9
the-most-h     8
mutual-aid     8
land           8
self-compa     8
greek-less     8
the-only-w     8
tomorrow,-     7
an-absolut     7
sisters-of     7
bowling-al     7
gideon-the     7
exit-inter     6
wanderers      6
to-shape-a     5
harry's-tr     5
we-came-to     5
lexicon        5
hannah-cou     4
the-three-     4
dtype: int64


In [93]:
# Get the top 10 books based on number votes above 4
top_10_winners = votes_above_4.head(10)
print("Top 10 winners:\n", top_10_winners)

# Where in spreadsheet to print the results
RESULTS_RANGE = 'results!A1:B11'

# Add column headers, reset the index, convert to a list of lists (required by the Sheets API)
values = [['Book title', 'Number of Votes Above 4']] + top_10_winners.reset_index().values.tolist()
body = {
    'values': values
}

# Use the Sheets API (authenticated above) to update the Google Sheet
result = sheet.values().update(
    spreadsheetId=SPREADSHEET_ID,
    range=RESULTS_RANGE,
    valueInputOption='RAW',
    body=body
).execute()

print(f"{result.get('updatedCells')} cells updated.")

Top 10 winners:
 the-upswin    10
women!-in!    10
apocalypti    10
algorithms     9
on-tyranny     9
exhalation     9
playground     9
the-most-h     8
mutual-aid     8
land           8
dtype: int64
22 cells updated.


In [94]:
# Clean up the results sheet
# Using the first column, look up the book title from the recs sheet
# and replace it in the results sheet

# Get the book titles from the recs sheet
BOOK_TITLES = 'recs!B2:B27'
result = sheet.values().get(
    spreadsheetId=SPREADSHEET_ID,
    range=BOOK_TITLES).execute()
books = result.get('values', [])
# Flatten
books = [book[0] for book in books]

for i, winner in enumerate(top_10_winners.index):
    # Replace dashes with spaces
    winner_cleaned = winner.replace('-', ' ')
    # Find the matching book title
    matching_title = next((book for book in books if winner_cleaned in book.lower()), None)
    
    if matching_title:
        # Add the book title to the results sheet
        result = sheet.values().update(
            spreadsheetId=SPREADSHEET_ID,
            range=f"results!A{i+2}",
            valueInputOption='RAW',
            body={'values': [[matching_title]]}
        ).execute()
        print(f"Added {matching_title} to the results sheet.")
        print(f"{result.get('updatedCells')} cells updated.")
    else:
        print(f"No matching title found for {winner}.")

Added The Upswing to the results sheet.
1 cells updated.
Added Women! In! Peril! to the results sheet.
1 cells updated.
Added Apocalyptic Planet to the results sheet.
1 cells updated.
Added Algorithms to live by to the results sheet.
1 cells updated.
Added On Tyranny  to the results sheet.
1 cells updated.
Added Exhalation  to the results sheet.
1 cells updated.
Added Playground to the results sheet.
1 cells updated.
Added The Most Human Human to the results sheet.
1 cells updated.
Added Mutual Aid to the results sheet.
1 cells updated.
Added Land to the results sheet.
1 cells updated.
